In [1]:
import rust_machine_learning
from syed_ml_lib import *
import os

In [71]:
# Setup the CNN
num_classifications = 10
max_pool_stride = 0
input_width = 64
input_height = 64
input_depth = 1
filters_per_conv_layer = [10,10]
filter_dimension = 3

CNN = rust_machine_learning.ConvolutionalNeuralNetwork(num_classifications, max_pool_stride, input_width, input_height, input_depth, filters_per_conv_layer, filter_dimension)

image_directory = "../data/animals-10/"
image_directory_path = os.path.abspath(image_directory)
CNN.set_image_loader(image_directory_path, input_width, input_height)

Reading images for classification "pecora"
Reading images for classification "cane"
Reading images for classification "ragno"
Reading images for classification "cavallo"
Reading images for classification "gallina"
Reading images for classification "scoiattolo"
Reading images for classification "farfalla"
Reading images for classification "gatto"
Reading images for classification "elefante"
Reading images for classification "mucca"


In [3]:
# Grab mnist data
import random
import numpy as np

df = pd.read_csv("../data/digit-recognizer/train.csv")

# Normalize images
df.iloc[:, 1:] = df.iloc[:, 1:].div(255.0).sub(0.5)
images = df.iloc[:, 1:].values.tolist()
df.iloc[:, 0] = df.iloc[:, 0].astype(np.float32)
labels = df.iloc[:, 0].values.tolist()

# Split into training and testing
test_images = images[0:20000]
test_labels = labels[0:20000]
train_images = images[20000:]
train_labels = labels[20000:]

df

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1.0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
1,0.0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
2,1.0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
3,4.0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
4,0.0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0.0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
41996,1.0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
41997,7.0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5
41998,6.0,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,...,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5,-0.5


In [4]:
animal_image_loader = rust_machine_learning.ImageBatchLoader(image_directory_path, input_width, input_height)
def test_animals():
    num_batches_to_test = 100
    num_right = 0
    for i in range(num_batches_to_test):

        images, label = animal_image_loader.batch_sample(1)

        prediction = CNN.classify(images)[0]
        if prediction == label[0]:
            num_right += 1

    print("Accuracy: " + str(100.0 * num_right / (num_batches_to_test)))

Reading images for classification "pecora"
Reading images for classification "cane"
Reading images for classification "ragno"
Reading images for classification "cavallo"
Reading images for classification "gallina"
Reading images for classification "scoiattolo"
Reading images for classification "farfalla"
Reading images for classification "gatto"
Reading images for classification "elefante"
Reading images for classification "mucca"


In [5]:
def test_mnist():
    num_batches_to_test = 100
    num_right = 0
    for i in range(num_batches_to_test):

        # Random sample one image from python list of images
        rand_index = random.randint(0, len(test_images) - 1)
        image = test_images[rand_index]
        label = test_labels[rand_index]

        prediction = CNN.classify([[image]])[0]
        if prediction == label:
            num_right += 1

    print("Accuracy: " + str(100.0 * num_right / (num_batches_to_test)))

In [6]:
def train_using_set_img_loader():
    learning_rate = 1e-5
    batch_size = 1
    num_iter = 1000
    print("Training...")
    CNN.train_using_image_loader(learning_rate, batch_size, num_iter)
    test_animals()

In [7]:
def train_using_raw_data():
    learning_rate = 1e-5
    batch_size = 10
    num_iter = 5000

    print("Training...")
    for i in range(0, num_iter):
        rand_index = random.randint(0, len(train_images) - 1)
        image = train_images[rand_index]
        label = train_labels[rand_index]

        # Train using MNIST dataset, select one image at a time randomly
        CNN.train_raw_data([[image]], [label], learning_rate)

    test_mnist()

In [8]:
# Now train the model
train_using_set_img_loader()

Training...
Accuracy: 14.0


In [70]:
# Interactive testing (for now animal only)
images, label = animal_image_loader.batch_sample(1)

prediction = CNN.classify(images)
prediction = prediction[0]

# Assert animal image loader and CNN are in sync
assert(animal_image_loader.get_classifications_map() == CNN.get_image_loader_classification_map())

# Now reverse mapping from string->float to float->string
classification_map = {}
for key, value in animal_image_loader.get_classifications_map().items():
    classification_map[value] = key

print("Label: " + classification_map[label[0]] + " " + str(label[0]))
print("Prediction: " + classification_map[prediction] + " (" + str(prediction) + ")")
view_image(images[0], input_width, input_height, colorscale="")

Label: cane 1.0
Prediction: gallina (4.0)
